In [3]:
import time
import torch
import numpy as np
from train_eval import train, init_network
from importlib import import_module
import argparse
dataset = 'THUCNews'  # 数据集

# 搜狗新闻:embedding_SougouNews.npz, 腾讯:embedding_Tencent.npz, 随机初始化:random
embedding = 'embedding_SougouNews.npz'

model_name = 'TextRCNN'  # 'TextRCNN'  # TextCNN, TextRNN, FastText, TextRCNN, TextRNN_Att, DPCNN, Transformer

from utils import build_dataset, build_iterator, get_time_dif

x = import_module('models.' + model_name)
config = x.Config(dataset, embedding)
vocab, train_data, dev_data, test_data = build_dataset(config, False)
train_iter = build_iterator(train_data, config)

Vocab size: 4762


180000it [00:01, 120957.85it/s]
10000it [00:00, 129748.16it/s]
10000it [00:00, 57614.94it/s]


In [6]:
train_iter.__next__()[1]

tensor([7, 8, 4, 3, 5, 5, 1, 0, 2, 5, 8, 4, 6, 7, 5, 0, 0, 7, 8, 1, 7, 8, 8, 9,
        1, 3, 0, 3, 5, 0, 1, 8, 6, 1, 7, 9, 2, 3, 4, 0, 7, 9, 5, 7, 9, 0, 1, 6,
        3, 6, 7, 9, 3, 6, 1, 8, 6, 8, 1, 2, 9, 2, 7, 9, 9, 7, 7, 6, 8, 8, 6, 8,
        7, 8, 3, 4, 3, 3, 2, 0, 0, 9, 6, 8, 6, 5, 4, 7, 6, 7, 8, 1, 8, 8, 5, 9,
        9, 6, 9, 1, 5, 4, 9, 1, 3, 6, 7, 1, 0, 0, 8, 6, 5, 8, 1, 4, 0, 2, 0, 8,
        5, 3, 3, 9, 9, 6, 4, 8])

In [3]:
config = x.Config(dataset, embedding)
np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
torch.backends.cudnn.deterministic = True  # 保证每次结果一样

In [5]:
config.pad_size = 56

In [49]:
# 假设你有一个列表
my_list = ['价格', '配置', '操控', '舒适性', '油耗', '动力', '内饰', '安全性', '空间', '外观']

# 指定文件名
filename = 'Newtrain/data/class.txt'

# 打开文件以写入模式 ('w' 表示写入，如果文件已存在则会被覆盖)
with open(filename, 'w') as file:
    # 遍历列表，将每个元素写入文件，每个元素后跟换行符('\n')
    for element in my_list:
        file.write(element + '\n')

print(f"List elements have been saved to {filename}")

List elements have been saved to Newtrain/data/class.txt


In [5]:
print("Loading data...")
vocab, train_data, dev_data, test_data = build_dataset(config, False)

Loading data...
Vocab size: 4762


180000it [00:01, 114935.30it/s]
10000it [00:00, 131912.53it/s]
10000it [00:00, 51791.44it/s]


In [ ]:
embeddings = np.random.rand(len(vocab), 300)
pretrain_dir = "THUCNews/data/sgns.sogou.char"
f = open(pretrain_dir, "r", encoding='UTF-8')
for i, line in enumerate(f.readlines()):
    
    # if i == 0:  # 若第一行是标题，则跳过
    #     continue
    lin = line.strip().split(" ")
    print(lin)
    if lin[0] in vocab:
        idx = vocab[lin[0]]
        emb = [float(x) for x in lin[1:301]]
        embeddings[idx] = np.asarray(emb, dtype='float32')
f.close()

In [8]:
import bz2
import os

def decompress_bz2(file_path):
    # 获取文件的基本路径和名称，用于构造解压后的文件名
    base_path, filename = os.path.split(file_path)
    output_file = os.path.join(base_path, filename[:-4])  # 移除'.bz2'扩展名
    
    with bz2.open(file_path, 'rb') as f_in:
        with open(output_file, 'wb') as f_out:
            data = f_in.read()
            f_out.write(data)
    
    print(f"File '{file_path}' has been successfully decompressed to '{output_file}'.")

# 指定你的.bz2文件路径
bz2_file_path = 'Newtrain/data/sgns.sogou.char.bz2'

# 调用函数解压文件
decompress_bz2(bz2_file_path)

File 'Newtrain/data/sgns.sogou.char.bz2' has been successfully decompressed to 'Newtrain/data/sgns.sogou.char'.


In [51]:
from sklearn.datasets import make_multilabel_classification
# 生成多标签数据集
X, Y = make_multilabel_classification(n_samples=10, n_features=20, n_classes=10, random_state=42)
X,Y

(array([[ 2.,  2.,  0.,  1.,  3.,  5.,  0.,  3.,  4.,  1.,  2.,  5.,  2.,
          1.,  4.,  1.,  3.,  4., 10.,  2.],
        [ 4.,  3.,  2.,  2.,  0.,  4.,  1.,  2.,  0.,  3.,  1.,  7.,  4.,
          2.,  3.,  1.,  2.,  2.,  2.,  1.],
        [ 2.,  2.,  3.,  0.,  0.,  0.,  0.,  6.,  0.,  3.,  4.,  0.,  5.,
          1.,  0.,  0.,  1.,  2.,  4.,  0.],
        [ 0.,  1.,  4.,  4.,  2.,  0.,  2.,  1.,  3.,  2.,  1.,  1.,  3.,
          0.,  0.,  2.,  6.,  3.,  3.,  1.],
        [ 0.,  0.,  7.,  2.,  1.,  0.,  1.,  2.,  1.,  2.,  2.,  1.,  4.,
          0.,  5.,  5.,  0.,  0.,  4.,  2.],
        [ 5.,  2.,  0.,  1.,  2.,  3.,  1.,  3.,  1.,  3.,  2.,  4.,  4.,
          2.,  2.,  0.,  3.,  2.,  5., 10.],
        [ 3.,  1.,  0.,  0.,  5.,  4.,  0.,  2.,  6.,  3.,  7.,  2.,  0.,
          4.,  2.,  1.,  4.,  6.,  6.,  2.],
        [ 5.,  2.,  1.,  4.,  3.,  3.,  0.,  3.,  0.,  7.,  2.,  2.,  1.,
          2.,  2.,  2.,  0.,  0.,  4.,  0.],
        [ 2.,  2.,  1.,  1.,  5.,  5.,  0.,  0.,

In [22]:
from utils import build_dataset, build_iterator, get_time_dif, build_vocab
import os
import torch
import numpy as np
import pickle as pkl
from tqdm import tqdm
import time
from datetime import timedelta
MAX_VOCAB_SIZE = 20000  # 词表长度限制
UNK, PAD = '<UNK>', '<PAD>'  # 未知字，padding符号
'''提取预训练词向量'''
# 下面的目录、文件名按需更改。
train_dir = "Newtrain/data/train.txt"
vocab_dir = "Newtrain/data/vocab.pkl"
pretrain_dir = "Newtrain/data/sgns.sogou.char"
emb_dim = 300
filename_trimmed_dir = "Newtrain/data/embedding_Newtrain_" + str(emb_dim)
if os.path.exists(vocab_dir):
    word_to_id = pkl.load(open(vocab_dir, 'rb'))
else:
    tokenizer = lambda x: x.split(' ')  # 以词为单位构建词表(数据集中词之间以空格隔开)
    # tokenizer = lambda x: [y for y in x]  # 以字为单位构建词表
    word_to_id = build_vocab(train_dir, tokenizer=tokenizer, max_size=MAX_VOCAB_SIZE, min_freq=1)
    pkl.dump(word_to_id, open(vocab_dir, 'wb'))

embeddings = np.random.rand(len(word_to_id), emb_dim)
f = open(pretrain_dir, "r", encoding='UTF-8')
for i, line in enumerate(f.readlines()):
    # if i == 0:  # 若第一行是标题，则跳过
    #     continue
    lin = line.strip().split(" ")
    if lin[0] in word_to_id:
        idx = word_to_id[lin[0]]
        emb = [float(x) for x in lin[1:emb_dim+1]]
        embeddings[idx] = np.asarray(emb, dtype='float32')
f.close()
np.savez_compressed(filename_trimmed_dir, embeddings=embeddings)

In [ ]:
word_to_id

In [ ]:
def build_dataset_(config, ues_word):
    if ues_word:
        tokenizer = lambda x: x.split(' ')  # 以空格隔开，word-level
    else:
        tokenizer = lambda x: [y for y in x]  # char-level
    if os.path.exists(config.vocab_path):
        vocab = pkl.load(open(config.vocab_path, 'rb'))
    else:
        vocab = build_vocab(config.train_path, tokenizer=tokenizer, max_size=MAX_VOCAB_SIZE, min_freq=1)
        pkl.dump(vocab, open(config.vocab_path, 'wb'))
    print(f"Vocab size: {len(vocab)}")

    def load_dataset(path, pad_size=32):
        contents = []
        with open(path, 'r', encoding='UTF-8') as f:
            for line in tqdm(f):
                lin = line.strip()
                if not lin:
                    continue
                content, label = lin.split('\t')
                words_line = []
                token = tokenizer(content)
                seq_len = len(token)
                if pad_size:
                    if len(token) < pad_size:
                        token.extend([PAD] * (pad_size - len(token)))
                    else:
                        token = token[:pad_size]
                        seq_len = pad_size
                # word to id
                for word in token:
                    words_line.append(vocab.get(word, vocab.get(UNK)))
                contents.append((words_line, int(label), seq_len))
        return contents  # [([...], 0), ([...], 1), ...]
    train = load_dataset(config.train_path, config.pad_size)
    dev = load_dataset(config.dev_path, config.pad_size)
    test = load_dataset(config.test_path, config.pad_size)
    return vocab, train, dev, test


In [30]:
str_list = ["1", "2", "3", "4"]
int_list = list(map(int, str_list))
int_list

[1, 2, 3, 4]

In [44]:
contents = []
pad_size = 64
from collections import defaultdict

with open(train_dir, 'r', encoding='UTF-8') as f:
    for line in tqdm(f):
        lin = line.strip()
        if not lin:
            continue
        tmp_data = lin.split('\t')
        content = tmp_data[0]
        zhuti_label = [t.split('#')[0] for t in tmp_data[1:]]

        cal_qinggan = sum([int(t.split('#')[1]) for t in tmp_data[1:]])
        qinggan_label = 0
        if cal_qinggan > 0:
            qinggan_label = 1
        elif cal_qinggan < 0:
            qinggan_label = -1

        words_line = []
        token = tokenizer(content)
        seq_len = len(token)
        if pad_size:
            if len(token) < pad_size:
                token.extend([PAD] * (pad_size - len(token)))
            else:
                token = token[:pad_size]
                seq_len = pad_size
        # word to id
        for word in token:
            words_line.append(word_to_id.get(word, word_to_id.get(UNK)))
        contents.append((words_line, zhuti_label, int(qinggan_label), seq_len))

8000it [00:00, 19987.38it/s]


In [45]:
k,q

(defaultdict(int,
             {'价格': 1237,
              '配置': 812,
              '操控': 986,
              '舒适性': 916,
              '油耗': 1043,
              '动力': 2643,
              '内饰': 515,
              '安全性': 537,
              '空间': 433,
              '外观': 479}),
 defaultdict(int, {0: 5660, -1: 1196, 1: 1144}))

In [48]:
print(k.keys())
class_name = ['价格', '配置', '操控', '舒适性', '油耗', '动力', '内饰', '安全性', '空间', '外观']

dict_keys(['价格', '配置', '操控', '舒适性', '油耗', '动力', '内饰', '安全性', '空间', '外观'])


In [85]:
z = []
for tmp in [[1,5], [2, 3], [0], [2]]:
    z.append(torch.sum(F.one_hot(torch.tensor(tmp), 10), axis=0))

In [86]:
torch.stack(z)

tensor([[0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [72]:
import torch
import torch.nn.functional as F
# 假设labels现在是类别索引，比如[0, 1, 2]表示第0、1、2个标签存在
labels_indices = torch.tensor([[0, 5], [1, 2], [0]], dtype=torch.long)  # 示例索引表示的标签
num_classes = 10 # 假设有3个可能的标签类别

# 转换为one-hot编码
one_hot_labels = F.one_hot(labels_indices, num_classes)
print(one_hot_labels)
torch.sum(one_hot_labels, axis=1)

ValueError: expected sequence of length 2 at dim 1 (got 1)

In [60]:
import torch
import torch.nn as nn
from torch.autograd import Variable

# 假设我们有3个类别，每个样本可以属于这些类别的任意子集
num_classes = 10
batch_size = 3

# 随机生成一些数据和目标标签（假设已经进行了适当的预处理）
inputs = torch.randn(batch_size, num_classes)  # 模型的输入logits
targets = torch.bernoulli(torch.rand(batch_size, num_classes))  # 生成0或1的目标标签，模拟多标签情况
print(targets)
# 将数据包装为Variable，如果使用的是PyTorch的较新版本，这一步通常不需要
inputs = Variable(inputs)
targets = Variable(targets)

# 定义模型，这里简化处理，直接使用inputs作为模型输出
model_output = inputs

# 定义损失函数
criterion = nn.BCEWithLogitsLoss()
print(model_output)
# 计算损失
loss = criterion(model_output, targets)

print(f"Loss: {loss.item()}")

tensor([[0., 1., 1., 1., 1., 1., 0., 1., 0., 0.],
        [1., 1., 1., 0., 0., 0., 1., 1., 0., 0.],
        [0., 0., 1., 0., 0., 1., 1., 1., 0., 0.],
        [1., 0., 0., 1., 0., 1., 0., 0., 1., 0.],
        [1., 0., 1., 0., 1., 0., 1., 1., 0., 1.]])
tensor([[-3.5894e-01,  4.7935e-01, -2.3828e-01, -1.3542e+00,  2.6869e-01,
          1.1456e-01, -1.5563e+00, -1.0757e+00, -5.7266e-02,  2.2000e+00],
        [ 9.9124e-01, -5.8622e-02,  1.1788e+00,  6.2218e-01,  7.8785e-01,
          1.3686e+00, -1.2211e+00, -6.7527e-01,  1.0476e+00, -3.1758e-01],
        [ 1.3949e-01,  2.3403e+00, -6.1161e-01,  8.1603e-01,  6.7713e-01,
          6.8086e-01,  1.9948e-01,  7.9927e-01, -2.6190e-01,  1.5133e-01],
        [ 1.1982e+00, -2.2833e+00,  3.3981e-01, -1.1494e+00, -6.0679e-01,
         -5.2522e-01, -5.6619e-01,  6.6040e-04,  7.2246e-01,  1.5264e-01],
        [ 1.4496e-01, -2.3442e+00,  3.6000e-01,  4.6668e-01,  1.2831e+00,
          1.2678e+00,  1.9883e-01,  5.4409e-01, -3.9782e-01, -1.9291e+00]])
Los